In [ ]:
#!pip install azure-cognitiveservices-vision-customvision msrest

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# ✅ Azure Custom Vision 설정 (API 키 및 엔드포인트 입력 필요)
prediction_endpoint = "<YOUR PREDICTION ENDPOINT>"
prediction_key = "YOUR PREDICTION KEY"

# ✅ API 인증 설정
credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

def first(image_data):
    """1차 분류: 과일/채소 판별"""
    project_id = "1-1 MODEL PROJECT ID"
    model_name = "1-1 MODEL NAME"

    results = predictor.classify_image(project_id, model_name, image_data)
    highest_prediction = max(results.predictions, key=lambda p: p.probability)

    return highest_prediction.tag_name, highest_prediction.probability * 100

def fresh(firstanswer, image_data):
    """2차 분류: 신선도 판별 (FRESH / MILD / ROTTEN)"""
    firstanswer = firstanswer.lower()  
    if firstanswer == "vegetable":
        project_id = "2-1 MODEL PROJECT ID"  # 채소 신선도 분류 모델
        model_name = "2-1 MODEL NAME"
    elif firstanswer == "fruits":
        project_id = "2-2 MODEL PROJECT ID"  # 과일 신선도 분류 모델
        model_name = "2-2 MODEL NAME"
    else:
        return None, 0  
    
    results = predictor.classify_image(project_id, model_name, image_data)
    fresh_prob, rotten_prob = 0, 0

    for prediction in results.predictions:
        if prediction.tag_name.lower() == "fresh":
            fresh_prob = prediction.probability * 100
        elif prediction.tag_name.lower() == "rotten":
            rotten_prob = prediction.probability * 100

    if fresh_prob >= 80:
        freshness = "FRESH"
    elif fresh_prob >= 50:
        freshness = "MILD"
    else:
        freshness = "ROTTEN"

    return freshness, fresh_prob, rotten_prob

def classify(firstanswer, image_data):
    """3차 분류: 과일/채소 종류 분류"""
    firstanswer = firstanswer.lower()  
    if firstanswer == "vegetable":
        project_id = "3-1 MODEL PROJECT ID"  # 채소 종류 분류 모델
        model_name = "3-1 MODEL NAME"
    elif firstanswer == "fruits":
        project_id = "3-2 MODEL PROJECT ID"  # 과일 종류 분류 모델
        model_name = "3-2 MODEL NAME"
    else:
        return None, 0  
    
    results = predictor.classify_image(project_id, model_name, image_data)
    highest_prediction = max(results.predictions, key=lambda p: p.probability)

    return highest_prediction.tag_name, highest_prediction.probability * 100

# ✅ 실행 코드 (이미지 로드 및 분류 실행)
image_path = r"IMAGE PATH"

with open(image_path, "rb") as image_data:
    image_bytes = image_data.read()  # ✅ 파일을 바이트 데이터로 읽기

firstanswer, first_prob = first(image_bytes)
print(f"1차 분류 결과: {firstanswer} ({first_prob:.2f}%)")

freshness, fresh_prob, rotten_prob = fresh(firstanswer, image_bytes)
type_name, type_prob = classify(firstanswer, image_bytes)

print(f"신선도 판별: {freshness} ({fresh_prob:.2f}% FRESH, {rotten_prob:.2f}% ROTTEN)")
print(f"종류 분류: {type_name} ({type_prob:.2f}%)")

# ✅ PIL을 이용한 이미지 출력
try:
    pil_image = Image.open(image_path)
    image = np.array(pil_image)

    plt.figure(figsize=(8, 6))  # 그림 크기 설정
    plt.imshow(image)  # 이미지 표시
    plt.axis("off")  # 축 제거
    plt.show()

except Exception as e:
    print(f"❌ PIL 이미지 로드 실패: {e}")

